In [1]:
import calendar
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import locale
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules,apriori
from scipy import sparse

In [2]:
course = pd.read_csv('Coursera_courses.csv')

course.head(3)

,name,institution,course_url,course_id
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning
1,Indigenous Canada,University of Alberta,https://www.coursera.org/learn/indigenous-canada,indigenous-canada
2,The Science of Well-Being,Yale University,https://www.coursera.org/learn/the-science-of-...,the-science-of-well-being


In [3]:
review = pd.read_csv('Coursera_reviews.csv')

review.head(3)

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training


In [4]:
df = pd.merge(course,review, on ='course_id')

df.head()

,name,institution,course_url,course_id,reviews,reviewers,date_reviews,rating
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,This is an extremely basic course. Machine lea...,By Deleted A,"Mar 18, 2017",1
1,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,The course is ok but the certification procedu...,By Bruno C,"Nov 09, 2015",1
2,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,"I just started week 3 , I have to admit that I...",By Fadi,"Apr 15, 2019",1
3,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,This course is absolute garbage. You get no f...,By Mathew L,"Sep 25, 2015",1
4,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,"However good the material and lectures may be,...",By Rui C,"Dec 12, 2015",1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1454711 entries, 0 to 1454710
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   name          1454711 non-null  object
 1   institution   1454711 non-null  object
 2   course_url    1454711 non-null  object
 3   course_id     1454711 non-null  object
 4   reviews       1454571 non-null  object
 5   reviewers     1454711 non-null  object
 6   date_reviews  1454711 non-null  object
 7   rating        1454711 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 99.9+ MB


In [6]:
df.reviewers.value_counts()

By Deleted A        5412
By Muhammad A        649
By Abhishek S        440
By David M           435
By Michael S         431
                    ... 
By Nwanja S            1
By Sabiha T            1
By Daniel A M G        1
By Lisa K F P          1
By América E C M       1
Name: reviewers, Length: 287808, dtype: int64

In [7]:
df = df[df["reviewers"] != 'By Deleted A']

In [8]:
pd.DataFrame(df.reviewers.value_counts()).head()


,reviewers
By Muhammad A,649
By Abhishek S,440
By David M,435
By Michael S,431
By Michael M,386


In [9]:
counts = df['reviewers'].value_counts()

to_remove = counts[counts < 100].index.tolist()

df.drop(df[df['reviewers'].isin(to_remove)].index, inplace=True)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56386 entries, 154 to 1454692
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          56386 non-null  object
 1   institution   56386 non-null  object
 2   course_url    56386 non-null  object
 3   course_id     56386 non-null  object
 4   reviews       56380 non-null  object
 5   reviewers     56386 non-null  object
 6   date_reviews  56386 non-null  object
 7   rating        56386 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 3.9+ MB


In [11]:
# df = df.sample(n=12850)


In [12]:
df.reviewers.value_counts()

By Muhammad A     649
By Abhishek S     440
By David M        435
By Michael S      431
By Michael M      386
                 ... 
By Sarah A        100
By Justin M       100
By Alexander T    100
By Rohit G        100
By Ian M          100
Name: reviewers, Length: 367, dtype: int64

In [13]:
pd.DataFrame(df.name.value_counts()).head(10)

,name
Machine Learning,2017
Programming for Everybody (Getting Started with Python),2015
Technical Support Fundamentals,1889
Neural Networks and Deep Learning,1883
Python Data Structures,1486
Learning How to Learn: Powerful mental tools to help you master tough subjects,1355
What is Data Science?,957
Using Python to Access Web Data,944
"Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization",867
The Data Scientist’s Toolbox,827


In [14]:
item_count_pivot = df.pivot_table(index='reviewers',columns='name', values= 'rating',aggfunc='sum').fillna(0)
item_count_pivot[item_count_pivot > 0] =1
item_count_pivot

name,A Crash Course in Causality: Inferring Causal Effects from Observational Data,A Law Student's Toolkit,AI For Everyone,AI for Medical Diagnosis,AI for Medical Prognosis,AWS Fundamentals: Addressing Security Risk,AWS Fundamentals: Going Cloud-Native,Aboriginal Worldviews and Education,Access Controls,Accounting Analytics,...,Wine Tasting: Sensory Techniques for Wine Analysis,Write A Feature Length Screenplay For Film Or Television,Write Professional Emails in English,Writing and Editing: Word Choice and Word Order,Writing in the Sciences,Étudier en France: French Intermediate course B1-B2,Математика и Python для анализа данных,Основы программирования на Python,Основы разработки на C++: белый пояс,Разработка веб-сервисов на Go - основы языка
reviewers,,,,,,,,,,,,,,,,,,,,,
By Abdullah A,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
By Abhinav S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
By Abhishek,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
By Abhishek A,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
By Abhishek B,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
By Vivek R,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
By Vivek S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
By William G,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Keterangan Olah Dataset
print('Dimensi Dataset :',item_count_pivot.shape)
print('Jumlah Reviewer User :',item_count_pivot.shape[0])
print('Jumlah Course :',item_count_pivot.shape[1])

Dimensi Dataset : (367, 564)
Jumlah Reviewer User : 367
Jumlah Course : 564


In [16]:
freq = apriori(item_count_pivot, min_support= 0.5, use_colnames= True)
freq.sort_values('support',ascending=False)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
2,0.795640,(Machine Learning)
3,0.782016,(Neural Networks and Deep Learning)
4,0.762943,(Programming for Everybody (Getting Started wi...
6,0.716621,(Technical Support Fundamentals)
5,0.694823,(Python Data Structures)
1,0.667575,(Learning How to Learn: Powerful mental tools ...
13,0.656676,"(Neural Networks and Deep Learning, Machine Le..."
17,0.637602,(Programming for Everybody (Getting Started wi...
14,0.626703,(Programming for Everybody (Getting Started wi...
16,0.588556,"(Technical Support Fundamentals, Machine Learn..."


In [17]:
rules = association_rules(freq,metric='lift',min_threshold=0.75)[['antecedents','consequents','support','confidence','lift']]
rules.sort_values(by=['support','confidence','lift'],ascending=False).head(30)

,antecedents,consequents,support,confidence,lift
6,(Neural Networks and Deep Learning),(Machine Learning),0.656676,0.839721,1.055403
7,(Machine Learning),(Neural Networks and Deep Learning),0.656676,0.825342,1.055403
14,(Programming for Everybody (Getting Started wi...,(Neural Networks and Deep Learning),0.637602,0.835714,1.068666
15,(Neural Networks and Deep Learning),(Programming for Everybody (Getting Started wi...,0.637602,0.815331,1.068666
8,(Programming for Everybody (Getting Started wi...,(Machine Learning),0.626703,0.821429,1.032412
9,(Machine Learning),(Programming for Everybody (Getting Started wi...,0.626703,0.787671,1.032412
12,(Technical Support Fundamentals),(Machine Learning),0.588556,0.821293,1.032241
13,(Machine Learning),(Technical Support Fundamentals),0.588556,0.739726,1.032241
20,(Python Data Structures),(Programming for Everybody (Getting Started wi...,0.572207,0.823529,1.079412
21,(Programming for Everybody (Getting Started wi...,(Python Data Structures),0.572207,0.750000,1.079412
